# Einsum

Einstein summation. pytorch, numpy, tensorflow 모두 지원. pytorch 에서는 조금 부족하다는 얘기도 있던데 확인.

Reference:

- https://ita9naiwa.github.io/numeric%20calculation/2018/11/10/Einsum.html (KR)
- https://rockt.github.io/2018/04/30/einsum (EN)

Matrix multiplication:
$$ \text{Result}=\text{einsum}(Bik,Bkj\rightarrow Bij)$$

- 기본적으론 위와 같은 형식. 즉, 겹치는 index 에 대해 연산이 이루어진다.

Outer product:
$$ \text{Result}=\text{einsum}(Bik,Bjk\rightarrow Bijk)$$

- 차원 수가 늘어나는 경우: outer product.
- 위는 i,j 가 outer product 로 결합되어 ij 가 되고 Bk 는 유지된 경우.

Hadamard product & Summation (=> Dot product):
$$ \text{Result}=\text{einsum}(Bik,Bjk\rightarrow Bij)$$

- k 가 겹치니까 k 에 대해서 hadamard product 연산이 들어가고, k 가 없어지므로 summation 이 들어간 것.
- 사실 matrix multiplication 과 hadamard product 가 완전히 달라 보이지만 einsum 의 관점에서 보면 그게 그거.
  - matrix multiplication 도 그냥 서로다른 축에 대해 hadamard product 가 수행되고 summation 이 수행된것.

## Excercise

아래 내용을 보고 올 것

```py
torch.einsum('xhyk,bvk,bqk->bhvq', (A, B, C))
```

- LHS 에는 있지만 RHS 는 없는 xyk 는 summation 이 되어 사라짐. 다만 xy 는 제일 앞에 애만 있고, k 는 셋다 있으니까 k는 hadamard product 이후 summation.
- 나머지 bhvq 는 남겨짐.
    - b 가 있는 BC 서로 hadamard product 를 하고, A는 없으므로 outer product.
    - 나머지 hvq 는 서로 하나씩만 있으니 서로 다 outer product.
    
- 정리해보면, 매트릭스는 각 차원이 벡터이고, 따라서 매트릭스 연산은 벡터 연산으로 분해할 수 있음.
- 벡터 연산은 두 종류로, hadamard product 와 outer product 로 나눌 수 있다. (dot product 는 그냥 hadamard product + summation)
    - hadamard product + summation => dot product / matrix multiplication
        - `[A] * [A] = [A]` (hadamard product)
        - `sum([A] * [A]) = []` (dot product)
        - matrix multiplication 은 이 벡터 연산을 어떤 축으로 할 것인가에 대한 것일 뿐임.
    - 연산을 할때 서로 사이즈가 다르면, outer product 가 가능. 이렇게 되면 위에서는 차원이 보존되거나 작아졌는데 여기서는 오히려 증가한다. 
        - `[A] * [B] = [A, B]` (outer product)

In [1]:
import torch

In [2]:
# b=3, h=2, v=5, q=6.
# x = y = 1, k=4.
A = torch.rand(1, 2, 1, 4)
B = torch.rand(3, 5, 4)
C = torch.rand(3, 6, 4)
r = torch.einsum('xhyk,bvk,bqk->bhvq', A, B, C)
print(r.shape)

torch.Size([3, 2, 5, 6])


In [3]:
for b in range(3):
    for h in range(2):
        for v in range(5):
            for q in range(6):
                assert r[b, h, v, q] == (A[:, h, :, :] * B[b, v, :] * C[b, q, :]).sum()

## Tutorial

In [4]:
A = torch.arange(2*3*4).reshape(2,3,4)
B = torch.arange(2*5*4).reshape(2,5,4)
torch.einsum("bik,bjk->bij", A, B)

tensor([[[  14,   38,   62,   86,  110],
         [  38,  126,  214,  302,  390],
         [  62,  214,  366,  518,  670]],

        [[1166, 1382, 1598, 1814, 2030],
         [1510, 1790, 2070, 2350, 2630],
         [1854, 2198, 2542, 2886, 3230]]])

In [5]:
import torch
import numpy as np

In [6]:
def print_einsum(cmd, *args, print_args=True):
    if print_args:
        print("args ({}):".format(len(args)))
        for v in args:
            print(v)
        print("")
    r = torch.einsum(cmd, *args)
    print("{}: {}".format(cmd, r.shape))
    print(r)

### Transpose

In [7]:
# Transpose
A = torch.arange(6).reshape(2, 3)
print_einsum("ij->ji", A)

args (1):
tensor([[0, 1, 2],
        [3, 4, 5]])

ij->ji: torch.Size([3, 2])
tensor([[0, 3],
        [1, 4],
        [2, 5]])


### Sum

In [8]:
# sum, column sum, row sum
A = torch.arange(6).reshape(2, 3)
print_einsum("ij->", A)
print_einsum("ij->j", A, print_args=False)
print_einsum("ij->i", A, print_args=False)

args (1):
tensor([[0, 1, 2],
        [3, 4, 5]])

ij->: torch.Size([])
tensor(15)
ij->j: torch.Size([3])
tensor([3, 5, 7])
ij->i: torch.Size([2])
tensor([ 3, 12])


### Multiplication

In [9]:
# matrix-vector multiplication
A = torch.arange(6).reshape(2, 3)
b = torch.arange(3)
print_einsum("ij,j->i", A, b)

args (2):
tensor([[0, 1, 2],
        [3, 4, 5]])
tensor([0, 1, 2])

ij,j->i: torch.Size([2])
tensor([ 5, 14])


In [10]:
# matrix-matrix multiplication
B = torch.arange(15).reshape(3, 5)
print_einsum("ik,kj->ij", A, B)
A @ B

args (2):
tensor([[0, 1, 2],
        [3, 4, 5]])
tensor([[ 0,  1,  2,  3,  4],
        [ 5,  6,  7,  8,  9],
        [10, 11, 12, 13, 14]])

ik,kj->ij: torch.Size([2, 5])
tensor([[ 25,  28,  31,  34,  37],
        [ 70,  82,  94, 106, 118]])


tensor([[ 25,  28,  31,  34,  37],
        [ 70,  82,  94, 106, 118]])

In [11]:
# batch matrix-matrix multiplication
A = torch.arange(2*3*4).reshape(2,3,4)
B = torch.arange(2*4*2).reshape(2,4,2)
print_einsum("ijv,ivk->ijk", A, B)
print(A @ B)
print(torch.bmm(A, B))

args (2):
tensor([[[ 0,  1,  2,  3],
         [ 4,  5,  6,  7],
         [ 8,  9, 10, 11]],

        [[12, 13, 14, 15],
         [16, 17, 18, 19],
         [20, 21, 22, 23]]])
tensor([[[ 0,  1],
         [ 2,  3],
         [ 4,  5],
         [ 6,  7]],

        [[ 8,  9],
         [10, 11],
         [12, 13],
         [14, 15]]])

ijv,ivk->ijk: torch.Size([2, 3, 2])
tensor([[[  28,   34],
         [  76,   98],
         [ 124,  162]],

        [[ 604,  658],
         [ 780,  850],
         [ 956, 1042]]])
tensor([[[  28,   34],
         [  76,   98],
         [ 124,  162]],

        [[ 604,  658],
         [ 780,  850],
         [ 956, 1042]]])
tensor([[[  28,   34],
         [  76,   98],
         [ 124,  162]],

        [[ 604,  658],
         [ 780,  850],
         [ 956, 1042]]])


### Dot product

In [12]:
# vector-vector dot product
a = torch.arange(3)
b = torch.arange(3, 6)
print_einsum("i,i->", a, b)
(a*b).sum()

args (2):
tensor([0, 1, 2])
tensor([3, 4, 5])

i,i->: torch.Size([])
tensor(14)


tensor(14)

In [13]:
# matrix-matrix dot product
A = torch.arange(6).reshape(2,3)
B = torch.arange(6, 12).reshape(2,3)
print_einsum("ij,ij->", A, B)
(A*B).sum()

args (2):
tensor([[0, 1, 2],
        [3, 4, 5]])
tensor([[ 6,  7,  8],
        [ 9, 10, 11]])

ij,ij->: torch.Size([])
tensor(145)


tensor(145)

### Hadamard product

Element-wise product

In [14]:
A = torch.arange(6).reshape(2,3)
B = torch.arange(6, 12).reshape(2,3)
print_einsum("ij,ij->ij", A, B)
A*B

args (2):
tensor([[0, 1, 2],
        [3, 4, 5]])
tensor([[ 6,  7,  8],
        [ 9, 10, 11]])

ij,ij->ij: torch.Size([2, 3])
tensor([[ 0,  7, 16],
        [27, 40, 55]])


tensor([[ 0,  7, 16],
        [27, 40, 55]])

### Outer product

In [15]:
a = torch.arange(3)
b = torch.arange(3, 7)
print_einsum("i,j->ij", a, b)
a.view(-1, 1) * b.view(1, -1)

args (2):
tensor([0, 1, 2])
tensor([3, 4, 5, 6])

i,j->ij: torch.Size([3, 4])
tensor([[ 0,  0,  0,  0],
        [ 3,  4,  5,  6],
        [ 6,  8, 10, 12]])


tensor([[ 0,  0,  0,  0],
        [ 3,  4,  5,  6],
        [ 6,  8, 10, 12]])

### Bilinear transformation

- 3개도 가능.

In [16]:
a = torch.randn(2,3)
b = torch.randn(5,3,7)
c = torch.randn(2,7)
torch.einsum('ik,jkl,il->ij', [a, b, c]).shape

torch.Size([2, 5])